In [1]:
import requests
import openai
import torch
from transformers import pipeline
import libs

In [2]:
libs.print_hello()

Hello my capivaras!


In [3]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds with jokes",
    },
    {
        "role": "user",
        "content": "How many helicopters can a human eat in one sitting?",
    },
]


In [4]:
from huggingface_hub import snapshot_download
model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0"
snapshot_download(
    repo_id=model_id,
    local_dir="/models/TinyLlama-1.1B-Chat-v1.0",
    local_dir_use_symlinks=False, 
    revision="main",
  )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1212: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]


KeyboardInterrupt



# Test TinyLlama

In [ ]:
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])
# <|system|>
# You are a friendly chatbot who always responds in the style of a pirate.</s>
# <|user|>
# How many helicopters can a human eat in one sitting?</s>
# <|assistant|>
# ...

# Test TinyLlama over LlamaCpp

In [5]:
response = requests.get(url='http://llm-server:8080/health')
response

<Response [200]>

In [6]:
# Not-streaming, chat completion, http client
response = requests.post(
    url='http://llm-server:8080/v1/chat/completions',
    headers={'Content-Type': 'application/json'},
    json={'messages': messages},
)
response.json()

{'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': 'There is no empirical evidence available that supports the claim that a human can eat in one sitting, but humans have been known to consume large amounts of food without any adverse effects. It is safe to assume that humans are capable of consuming enough food to sustain themselves in one sitting if they choose to do so. In general, the human body is designed to store and utilize nutrients efficiently, allowing it to consume and use food efficiently in times of food scarcity. However, this is a matter of scientific research and not a matter of personal opinion or belief.',
    'role': 'assistant'}}],
 'created': 1723547982,
 'model': 'gpt-3.5-turbo-0613',
 'object': 'chat.completion',
 'usage': {'completion_tokens': 118, 'prompt_tokens': 63, 'total_tokens': 181},
 'id': 'chatcmpl-3ml8PSkWTQAd1jW6X0cxuFxmfcQUA1CI'}

In [6]:
# Streaming, chat completion, http client
response = requests.post(
    url='http://llm-server:8080/v1/chat/completions',
    headers={'Content-Type': 'application/json'},
    json={'messages': messages, 'stream': True},
    stream=True,
)
for chunk in response:
    print(chunk[:25], end='')
    #print('---------')

b'data: {"choices":[{"finis'b'FHAOMYUbRs99w1YX","model"'b'data: {"choices":[{"finis'b'lFHAOMYUbRs99w1YX","model'b'data: {"choices":[{"finis'b'OlFHAOMYUbRs99w1YX","mode'b'data: {"choices":[{"finis'b'gOlFHAOMYUbRs99w1YX","mod'b'data: {"choices":[{"finis'b'MhzugOlFHAOMYUbRs99w1YX",'b'data: {"choices":[{"finis'b'hzugOlFHAOMYUbRs99w1YX","'b'data: {"choices":[{"finis'b'FHAOMYUbRs99w1YX","model"'b'data: {"choices":[{"finis'b'FHAOMYUbRs99w1YX","model"'b'data: {"choices":[{"finis'b'FHAOMYUbRs99w1YX","model"'b'data: {"choices":[{"finis'b'FHAOMYUbRs99w1YX","model"'b'data: {"choices":[{"finis'b'gOlFHAOMYUbRs99w1YX","mod'b'data: {"choices":[{"finis'b'OlFHAOMYUbRs99w1YX","mode'b'data: {"choices":[{"finis'b'lFHAOMYUbRs99w1YX","model'b'data: {"choices":[{"finis'b'OlFHAOMYUbRs99w1YX","mode'b'data: {"choices":[{"finis'b'gOlFHAOMYUbRs99w1YX","mod'b'data: {"choices":[{"finis'b'gOlFHAOMYUbRs99w1YX","mod'b'data: {"choices":[{"finis'b'FHAOMYUbRs99w1YX","model"'b'data: {"choices":[{"finis'b'9w1YX","model":"gp

In [7]:
client = openai.OpenAI(
    base_url="http://llm-server:8080/v1",
    api_key = "not-needed"
)

In [26]:
# Not-streaming, text completion, openai client
completion = client.completions.create(
    model="not-needed",
    prompt='The capital of france is',
    max_tokens = 5,
)
print(type(completion))
print(completion.content)

<class 'openai.types.completion.Completion'>
 Paris, located in the


In [20]:
# Not-streaming, chat completion, openai client
completion = client.chat.completions.create(
    model="not-needed",
    messages=messages,
)

print(completion.choices[0].message)

ChatCompletionMessage(content="It's not possible to eat helicopters in one sitting. Helicopters are small, flightless birds with a long, narrow, and blunt-ended tail. They have a bony, spiked bill and are primarily nocturnal predators that feed on insects, small reptiles, and amphibians.", refusal=None, role='assistant', function_call=None, tool_calls=None)


In [21]:
completion.dict()

{'id': 'chatcmpl-Hcqaaa0GZMWEvHAt9CVIVCvEBboaUrGb',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': "It's not possible to eat helicopters in one sitting. Helicopters are small, flightless birds with a long, narrow, and blunt-ended tail. They have a bony, spiked bill and are primarily nocturnal predators that feed on insects, small reptiles, and amphibians.",
    'refusal': None,
    'role': 'assistant',
    'function_call': None,
    'tool_calls': None}}],
 'created': 1723125684,
 'model': 'not-needed',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': None,
 'usage': {'completion_tokens': 75, 'prompt_tokens': 63, 'total_tokens': 138}}

In [22]:
# Streaming, text completion, openai client
completion = client.completions.create(
    model="not-needed",
    prompt='The capital of france is',
    max_tokens = 5,
    stream=True,
)
for chunk in completion:
    print(chunk.content, end='')

 a place of great cultural

In [23]:
type(completion)

openai.Stream

In [24]:
type(chunk)

openai.types.completion.Completion

In [25]:
chunk.dict()

{'id': None,
 'choices': None,
 'created': None,
 'model': '/root/.cache/llama.cpp/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf',
 'object': None,
 'system_fingerprint': None,
 'usage': None,
 'content': '',
 'id_slot': 0,
 'stop': True,
 'tokens_predicted': 5,
 'tokens_evaluated': 7,
 'generation_settings': {'n_ctx': 2048,
  'n_predict': -1,
  'model': '/root/.cache/llama.cpp/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf',
  'seed': 4294967295,
  'temperature': 0.800000011920929,
  'dynatemp_range': 0.0,
  'dynatemp_exponent': 1.0,
  'top_k': 40,
  'top_p': 0.949999988079071,
  'min_p': 0.05000000074505806,
  'tfs_z': 1.0,
  'typical_p': 1.0,
  'repeat_last_n': 64,
  'repeat_penalty': 1.0,
  'presence_penalty': 0.0,
  'frequency_penalty': 0.0,
  'penalty_prompt_tokens': [],
  'use_penalty_prompt_tokens': False,
  'mirostat': 0,
  'mirostat_tau': 5.0,
  'mirostat_eta': 0.10000000149011612,
  'penalize_nl': False,
  'stop': [],
  'n_keep': 0,
  'n_discard': 0,
  'ignore_eos': False,
  'stream': True,
  

In [12]:
# Streaming, chat completion, openai client
completion = client.chat.completions.create(
    model="not-needed",
    messages=messages,
    stream=True,
)

for chunk in completion:
    #print(chunk)
    print(chunk.choices[0].delta.content or '', end='')
    #print("****************")

A human can eat approximately 10 to 15 helicopters in one sitting. Helicopters are the popular snacks in certain cultures.

# Test TinyLlama over LlamaCpp over proxy

In [13]:
requests.get('http://orchestrator-proxy:80/docs')

<Response [200]>

In [4]:
# Streaming, chat completion, http client
# Works, but return all at once
response = requests.post(
    url='http://orchestrator-proxy:80/v1/chat/completions',
    headers={'Content-Type': 'application/json', 'Authorization': 'not-needed'},
    json={'messages': messages, 'stream': True},
    stream=True,
)
for chunk in response:
    print(chunk[:25], end='')
    #print('---------')

b'data: {"choices":[{"finis'b'rgdDLq8Yr0rgdgba0O","mode'b'data: {"choices":[{"finis'b'argdDLq8Yr0rgdgba0O","mod'b'data: {"choices":[{"finis'b'gdDLq8Yr0rgdgba0O","model'b'data: {"choices":[{"finis'b'eargdDLq8Yr0rgdgba0O","mo'b'data: {"choices":[{"finis'b'argdDLq8Yr0rgdgba0O","mod'b'data: {"choices":[{"finis'b'eargdDLq8Yr0rgdgba0O","mo'b'data: {"choices":[{"finis'b'rgdDLq8Yr0rgdgba0O","mode'b'data: {"choices":[{"finis'b'gdDLq8Yr0rgdgba0O","model'b'data: {"choices":[{"finis'b'QeargdDLq8Yr0rgdgba0O","m'b'data: {"choices":[{"finis'b'argdDLq8Yr0rgdgba0O","mod'b'data: {"choices":[{"finis'b'QeargdDLq8Yr0rgdgba0O","m'b'data: {"choices":[{"finis'b'dDLq8Yr0rgdgba0O","model"'b'data: {"choices":[{"finis'b'dDLq8Yr0rgdgba0O","model"'b'data: {"choices":[{"finis'b'dDLq8Yr0rgdgba0O","model"'b'data: {"choices":[{"finis'b'dDLq8Yr0rgdgba0O","model"'b'data: {"choices":[{"finis'b'QeargdDLq8Yr0rgdgba0O","m'b'data: {"choices":[{"finis'b'dDLq8Yr0rgdgba0O","model"'b'data: {"choices":[{"finis'b'QeargdDLq8Yr0rgdgb

In [12]:
type(client)

openai.OpenAI

In [7]:
client = openai.OpenAI(
    base_url="http://orchestrator-proxy:80/v1",
    api_key = "not-needed"
)

In [8]:
# Not-streaming, text completion, openai client
completion = client.completions.create(
    model="not-needed",
    prompt='I will answer this with one single word: The capital of france is',
    max_tokens = 5,
)

print(completion.content)

 not the capital of fran


In [17]:
# Not-streaming, chat completion, openai client
completion = client.chat.completions.create(
    model="not-needed",
    messages=messages,
)

print(completion.choices[0].message.content)

One human can consume approximately 20 helicopters (or 20,000 units) of helicoptical jelly in one sitting. The exact quantity depends on the human's appetite and the quantity of helicoptical jelly available.


In [27]:
# Streaming, text completion, openai client
# Works, but return all at once
completion = client.completions.create(
    model="not-needed",
    prompt='The capital of france is',
    max_tokens = 5,
    stream=True,
)
for chunk in completion:
    print(chunk.content, end='')

 the city of paris

In [19]:
# Streaming, chat completion, openai client
# Works, but return all at once
completion = client.chat.completions.create(
    model="not-needed",
    messages=messages,
    stream=True,
)

for chunk in completion:
    #print(chunk)
    print(chunk.choices[0].delta.content or '', end='')
    #print("****************")

I do not have access to accurate information about the specific number of helicopters that a human can consume in one sitting, as this information is usually provided by the manufacturer or restaurant. However, it is recommended to consume one helicoptor per serving (about 180-220g), which is equivalent to the standard serving size of around 1/2 cup (50g). So, if you consume 6 servings per day, you will have consumed around 300-360 helicoptors.